In [1]:
from os import listdir

from pandas import DataFrame, concat, read_csv

games = []
filenames = listdir("./data/raw")
current_df = None
for file in sorted(filenames, reverse=True):
    if ".csv" in file:
        current_df = read_csv("./data/raw/" + file)[
            10:
        ]  # each file has 10 repeating rows
        games.append(current_df[::-1])
        games_df = concat(games, ignore_index=True)
        games_df = games_df.rename(columns={"winner": "player", "loser": "opponent"})

        break

In [2]:
# Let each row with empty date have the same one from above
from numpy import NaN

games_df = games_df.replace({"\xa0": NaN})
games_df = games_df.fillna(method="backfill")

games_df

,date,player,opponent
0,2021.01.01,원성진,최 정
1,2021.01.01,신진서,한승주
2,2021.01.01,나 현,조한승
3,2021.01.01,이영구,이태현
4,2021.01.02,신민준,박건호
...,...,...,...
7705,2021.12.10,강지훈,백현우
7706,2021.12.11,변상일,쉬자위안
7707,2021.12.11,안성준,김정현
7708,2021.12.11,신진서,쉬자양


In [3]:
from tqdm import tqdm

"""
    Compute every player's cumulative streak
    
    PS = Player Streak
    OS = Opponenet Streak
"""

for player in tqdm(games_df.player.unique()):
    filtered_df = games_df.loc[
        (games_df.player == player) | (games_df.opponent == player)
    ]
    filtered_df = filtered_df.assign(player_result=lambda row: row.player == player)
    filtered_df = filtered_df.assign(
        player_start_of_streak=filtered_df.player_result.ne(
            filtered_df["player_result"].shift()
        )
    )
    filtered_df = filtered_df.assign(
        player_streak_id=filtered_df["player_start_of_streak"].cumsum()
    )
    filtered_df = filtered_df.assign(
        PS=filtered_df.groupby("player_streak_id").cumcount() + 1
    )

    filtered_df = filtered_df.assign(opponent_result=lambda row: row.opponent == player)
    filtered_df = filtered_df.assign(
        opponent_start_of_streak=filtered_df.opponent_result.ne(
            filtered_df["opponent_result"].shift()
        )
    )
    filtered_df = filtered_df.assign(
        opponent_streak_id=filtered_df["opponent_start_of_streak"].cumsum()
    )
    filtered_df = filtered_df.assign(
        OS=filtered_df.groupby("opponent_streak_id").cumcount() + 1
    )

    for index, row in filtered_df.iterrows():
        if row.player_result:
            games_df.loc[index, "PS"] = row.PS
        if row.opponent_result:
            games_df.loc[index, "OS"] = row.OS

games_df

100%|█████████████████████████████████████████| 450/450 [00:08<00:00, 53.07it/s]


,date,player,opponent,PS,OS
0,2021.01.01,원성진,최 정,1.0,1.0
1,2021.01.01,신진서,한승주,1.0,1.0
2,2021.01.01,나 현,조한승,1.0,1.0
3,2021.01.01,이영구,이태현,1.0,1.0
4,2021.01.02,신민준,박건호,1.0,1.0
...,...,...,...,...,...
7705,2021.12.10,강지훈,백현우,2.0,1.0
7706,2021.12.11,변상일,쉬자위안,5.0,1.0
7707,2021.12.11,안성준,김정현,3.0,1.0
7708,2021.12.11,신진서,쉬자양,12.0,1.0


In [10]:
from pandas import concat
from tqdm import tqdm

"""
    Compute every player's cumulative win rates and last 5 game status
    
    PWR = Player Win Rate
    OWR = Opponent Win Rate
    PL5G = Player Last 5 Game
    OL5G = Opponent Last 5 Game
"""

games_df["PWR"], games_df["OWR"] = NaN, NaN
games_df["PL5G"], games_df["OL5G"] = NaN, NaN

for player in tqdm(games_df.player.unique()):
    filtered_df = games_df.loc[
        (games_df.player == player) | (games_df.opponent == player),
        ["player", "opponent"],
    ]
    filtered_df.insert(
        filtered_df.shape[1], "num_games", range(1, len(filtered_df) + 1)
    )
    filtered_df = filtered_df.assign(player_win=lambda row: row.player == player)
    filtered_df = filtered_df.assign(opponent_lose=lambda row: row.opponent == player)

    filtered_df.loc[filtered_df["player"] == player, "PNW"] = filtered_df[
        "player_win"
    ].cumsum()
    filtered_df["PNL"] = filtered_df["num_games"] - filtered_df["PNW"]
    filtered_df.loc[filtered_df["opponent"] == player, "ONL"] = filtered_df[
        "opponent_lose"
    ].cumsum()
    filtered_df["ONW"] = filtered_df["num_games"] - filtered_df["ONL"]

    filtered_df["PWR"] = filtered_df["PNW"] / filtered_df["num_games"]
    filtered_df["OWR"] = filtered_df["ONW"] / filtered_df["num_games"]

    filtered_df.loc[filtered_df["player"] == player, "PL5G"] = (
        filtered_df["player_win"].rolling(5).sum()
        - filtered_df["opponent_lose"].rolling(5).sum()
    )
    filtered_df.loc[filtered_df["opponent"] == player, "OL5G"] = (
        filtered_df["player_win"].rolling(5).sum()
        - filtered_df["opponent_lose"].rolling(5).sum()
    )

    filtered_df = filtered_df.loc[:, ["PWR", "OWR", "PL5G", "OL5G"]]
    games_df.update(filtered_df)

games_df[(games_df["player"] == "이창석") | (games_df["opponent"] == "이창석")]

100%|█████████████████████████████████████████| 450/450 [00:05<00:00, 81.58it/s]


,date,player,opponent,PS,OS,PWR,OWR,PL5G,OL5G
5,2021.01.02,이창석,강동윤,1.0,1.0,1.000000,0.000000,NaN,NaN
57,2021.01.07,이창석,나 현,2.0,1.0,1.000000,0.500000,NaN,NaN
81,2021.01.10,이창석,설현준,3.0,2.0,1.000000,0.000000,NaN,NaN
180,2021.01.14,이창석,오유진,4.0,1.0,1.000000,0.250000,NaN,NaN
295,2021.01.17,신진서,이창석,5.0,1.0,1.000000,0.800000,5.0,3.0
...,...,...,...,...,...,...,...,...,...
7384,2021.11.29,홍무진,이창석,4.0,1.0,0.760000,0.679487,3.0,1.0
7421,2021.12.02,이창석,강승민,1.0,5.0,0.683544,0.500000,3.0,-5.0
7480,2021.12.06,이창석,김상인,2.0,2.0,0.687500,0.393617,3.0,-1.0
7566,2021.12.07,이창석,김승준,3.0,1.0,0.691358,0.458333,3.0,-1.0


In [4]:
# Add new column of win - lose of last 5 games

# iterate in reverse order and fill in hash table
# key = player name | value = [win_count, lose_count, last_five_games]
from tqdm import tqdm

QUEUE_SIZE = 5
table = {}


class Queue:
    def __init__(self, n=QUEUE_SIZE):
        self.queue = []
        self.max_capacity = n

    def push(self, val):
        if len(self.queue) == self.max_capacity:
            self.queue.pop(0)

        self.queue.append(val)

    def sum(self):
        return sum(self.queue)


filtered_df = DataFrame()

for player in tqdm(games_df.player.unique()):
    filtered_df = games_df.loc[
        (games_df.player == player) | (games_df.opponent == player)
    ]
for idx, row in tqdm(games_df[::-1].iterrows(), total=games_df.shape[0]):

    player = row["player"]
    opponent = row["opponent"]

    if player in table:
        table[player][0] += 1
        table[player][2].push(1)
    else:
        new_queue = Queue()
        new_queue.push(1)
        table[player] = [1, 0, new_queue]

    if opponent in table:
        table[opponent][1] += 1
        table[opponent][2].push(-1)
    else:
        new_queue = Queue()
        new_queue.push(-1)
        table[opponent] = [0, 1, new_queue]

    games_df[idx, "PL5G"] = table[player][2].sum()
    games_df[idx, "PNW"] = table[player][0]
    games_df[idx, "PNL"] = table[player][1]
    games_df[idx, "PWR"] = table[player][0] / (table[player][0] + table[player][1])
    games_df[idx, "OL5G"] = table[opponent][2].sum()
    games_df[idx, "ONW"] = table[opponent][0]
    games_df[idx, "ONL"] = table[opponent][1]
    games_df[idx, "OWR"] = table[opponent][0] / (
        table[opponent][0] + table[opponent][1]
    )

games_df
# for i, row1 in tqdm(games_df.iterrows(), total=games_df.shape[0]):
#     filtered_df = games_df.iloc[i + 1 :].loc[
#         (games_df["player"] == row1["player"]) | (games_df["opponent"] == row1["player"])
#     ][:5]
#     value = 0
#     for _, row2 in filtered_df.iterrows():
#         value = value + 1 if row1["player"] == row2["player"] else value - 1
#     games_df.loc[i, "PL5G"] = value

# # Add new column of win - lose of last 10 games
# for i, row1 in tqdm(games_df.iterrows(), total=games_df.shape[0]):
#     filtered_df = games_df.iloc[i + 1 :].loc[
#         (games_df["player"] == row1["opponent"]) | (games_df["opponent"] == row1["opponent"])
#     ][:5]
#     value = 0
#     for _, row2 in filtered_df.iterrows():
#         value = value + 1 if row1["opponent"] == row2["player"] else value - 1
#     games_df.loc[i, "OL5G"] = value

100%|███████████████████████████████████████| 7710/7710 [34:29<00:00,  3.73it/s]


,date,player,opponent,"(7709, PL5G)","(7709, PNW)","(7709, PNL)","(7709, PWR)","(7709, OL5G)","(7709, ONW)","(7709, ONL)",...,"(1, ONL)","(1, OWR)","(0, PL5G)","(0, PNW)","(0, PNL)","(0, PWR)","(0, OL5G)","(0, ONW)","(0, ONL)","(0, OWR)"
0,2021.01.01,원성진,최 정,1,1,0,1.0,-1,0,1,...,22,0.666667,5,31,18,0.632653,-1,68,28,0.708333
1,2021.01.01,신진서,한승주,1,1,0,1.0,-1,0,1,...,22,0.666667,5,31,18,0.632653,-1,68,28,0.708333
2,2021.01.01,나 현,조한승,1,1,0,1.0,-1,0,1,...,22,0.666667,5,31,18,0.632653,-1,68,28,0.708333
3,2021.01.01,이영구,이태현,1,1,0,1.0,-1,0,1,...,22,0.666667,5,31,18,0.632653,-1,68,28,0.708333
4,2021.01.02,신민준,박건호,1,1,0,1.0,-1,0,1,...,22,0.666667,5,31,18,0.632653,-1,68,28,0.708333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7705,2021.12.10,강지훈,백현우,1,1,0,1.0,-1,0,1,...,22,0.666667,5,31,18,0.632653,-1,68,28,0.708333
7706,2021.12.11,변상일,쉬자위안,1,1,0,1.0,-1,0,1,...,22,0.666667,5,31,18,0.632653,-1,68,28,0.708333
7707,2021.12.11,안성준,김정현,1,1,0,1.0,-1,0,1,...,22,0.666667,5,31,18,0.632653,-1,68,28,0.708333
7708,2021.12.11,신진서,쉬자양,1,1,0,1.0,-1,0,1,...,22,0.666667,5,31,18,0.632653,-1,68,28,0.708333


In [11]:
# Add new column of against
for i, row1 in tqdm(games_df.iterrows(), total=games_df.shape[0]):
    filtered_df1 = games_df.iloc[i + 1 :].loc[
        (
            (row1["player"] == games_df["player"])
            & (row1["opponent"] == games_df["opponent"])
        )
    ]
    filtered_df2 = games_df.iloc[i + 1 :].loc[
        (
            (row1["player"] == games_df["opponent"])
            & (row1["opponent"] == games_df["player"])
        )
    ]

    games_df.loc[i, "PAW"] = filtered_df1.shape[0]
    games_df.loc[i, "OAW"] = filtered_df2.shape[0]

 12%|████▊                                  | 959/7710 [00:04<00:33, 200.17it/s]


KeyboardInterrupt: 

In [ ]:
# Add new column of winner win percentage
for i, row1 in tqdm(games_df.iterrows(), total=games_df.shape[0]):
    filtered_df1 = games_df.iloc[i + 1 :].loc[(row1["player"] == games_df["player"])]
    filtered_df2 = games_df.iloc[i + 1 :].loc[(row1["player"] == games_df["opponent"])]

    games_df.loc[i, "PNW"] = filtered_df1.shape[0]
    games_df.loc[i, "PNL"] = filtered_df2.shape[1]

# Add new column of loser win percentage
for i, row1 in tqdm(games_df.iterrows(), total=games_df.shape[0]):
    filtered_df1 = games_df.iloc[i + 1 :].loc[(row1["opponent"] == games_df["player"])]
    filtered_df2 = games_df.iloc[i + 1 :].loc[
        (row1["opponent"] == games_df["opponent"])
    ]

    games_df.loc[i, "ONW"] = filtered_df1.shape[0]
    games_df.loc[i, "ONL"] = filtered_df2.shape[0]

    games_df["PWR"] = games_df["PNW"] / (games_df["PNW"] + games_df["PNL"])
    games_df["OWR"] = games_df["ONW"] / (games_df["ONW"] + games_df["ONL"])
    games_df["AWR"] = games_df["PAW"] / (games_df["PAW"] + games_df["OAW"])

print(games_df.head())

In [ ]:
games_df.to_csv("./data/processed_games.csv")